# Decision trees
- simple tree like structure,model makes a decision at every node
- useful in simple tasks
- one of the most popular algorithms
- easy explainability,easy to show how a decision process works
# why decision trees are popular
- easy to interpet and present
- well defined logic, mimic human level thought
- random forests,ensembles of decision trees are more powerfull classifiers
- feature values are preferred to be categorical.if the values are continuous then they are discretized prior to building the model
# build decision tress
- two common algorithms-
- CART (classifier and regression tree) -> uses gini index (classification) as metric
- ID3 (iterative dichotomiser 3) -> uses entropy function and information gain as metrics

# TITANIC SURVIVOR PREDICTION KAGGLE CHALLENGE

# Learning Goals 
- how to pre-process data?
- dropping not useful features
- filling the missiing values(data imputation)
- creating a binary decision tree from scratch

In [1]:
import numpy as np
import pandas as pd

In [2]:
train_x=pd.read_csv("train titanic.csv")
test_x=pd.read_csv("test titanic.csv")

In [3]:
train_x.head(n=10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


# data preparation

In [5]:
test_x.shape

(418, 11)

In [6]:
train_x.shape

(891, 12)

In [7]:
train_x.info() #it shows all detail of data

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


In [34]:
# here in above data we have some feature which are not useable for prediction so we drop that features
columns_to_drop=["PassengerId","Name","Ticket","Embarked","Cabin"]
train_x_clean=train_x.drop(columns_to_drop,axis=1)
train_x_clean.head(n=10)
test_x_clean=test_x.drop(columns_to_drop,axis=1)

In [35]:
# here sex is of type string we have to convert it into numeric type we use sklearn for it
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
train_x_clean["Sex"]=le.fit_transform(train_x_clean["Sex"])
train_x_clean.head(n=5)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare
0,0,3,1,22.0,1,0,7.2500
1,1,1,0,38.0,1,0,71.2833
2,1,3,0,26.0,0,0,7.9250
3,1,1,0,35.0,1,0,53.1000
4,0,3,1,35.0,0,0,8.0500


In [36]:
test_x_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 6 columns):
Pclass    418 non-null int64
Sex       418 non-null object
Age       332 non-null float64
SibSp     418 non-null int64
Parch     418 non-null int64
Fare      417 non-null float64
dtypes: float64(2), int64(3), object(1)
memory usage: 19.7+ KB


In [37]:
train_x_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 7 columns):
Survived    891 non-null int64
Pclass      891 non-null int64
Sex         891 non-null int32
Age         714 non-null float64
SibSp       891 non-null int64
Parch       891 non-null int64
Fare        891 non-null float64
dtypes: float64(2), int32(1), int64(4)
memory usage: 45.3 KB


In [38]:
test_x_clean["Sex"]=le.fit_transform(test_x_clean["Sex"])
test_x_clean=test_x_clean.fillna(test_x_clean["Age"].mean())

In [39]:
test_x_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 6 columns):
Pclass    418 non-null int64
Sex       418 non-null int32
Age       418 non-null float64
SibSp     418 non-null int64
Parch     418 non-null int64
Fare      418 non-null float64
dtypes: float64(2), int32(1), int64(3)
memory usage: 18.0 KB


In [40]:
test_x_clean.head(n=10)

,Pclass,Sex,Age,SibSp,Parch,Fare
0,3,1,34.5,0,0,7.8292
1,3,0,47.0,1,0,7.0000
2,2,1,62.0,0,0,9.6875
3,3,1,27.0,0,0,8.6625
4,3,0,22.0,1,1,12.2875
5,3,1,14.0,0,0,9.2250
6,3,0,30.0,0,0,7.6292
7,2,1,26.0,1,1,29.0000
8,3,0,18.0,0,0,7.2292
9,3,1,21.0,2,0,24.1500


# we can see above that all the feature are in numeric form

In [10]:
train_x_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 7 columns):
Survived    891 non-null int64
Pclass      891 non-null int64
Sex         891 non-null int32
Age         714 non-null float64
SibSp       891 non-null int64
Parch       891 non-null int64
Fare        891 non-null float64
dtypes: float64(2), int32(1), int64(4)
memory usage: 45.3 KB


- here we can see that all feature have 891 data but age feature have 714 data so we have to add some age value in missing data we add average value of age in all missing data

In [11]:
train_x_clean=train_x_clean.fillna(train_x_clean["Age"].mean())  # it fill all nan value of age column by mean of age of all example we can also use imputer from sklearn library

In [12]:
# imputer ----> sklearn !

In [13]:
input_cols=["Pclass","Sex","Age","SibSp","Parch","Fare"]
output_cols=["Survived"]
train_y1=train_x_clean[output_cols]
train_x1=train_x_clean[input_cols]

In [14]:
train_y1.shape,train_x1.shape,type(train_y1)

((891, 1), (891, 6), pandas.core.frame.DataFrame)

In [15]:
train_x1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 6 columns):
Pclass    891 non-null int64
Sex       891 non-null int32
Age       891 non-null float64
SibSp     891 non-null int64
Parch     891 non-null int64
Fare      891 non-null float64
dtypes: float64(2), int32(1), int64(3)
memory usage: 38.4 KB


# define entropy and information gain

In [16]:
def entropy(col):
    x=np.unique(col,return_counts=True)
    n=float(col.shape[0])
    e=0.0
    for ix in x[1]:
        p=float(ix/n)
        e+=(-1.0*p*np.log2(p))
    return e

In [17]:
col=np.array([1,1,1,0,0,0])
print(entropy(col))

1.0


In [18]:
train_x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


In [19]:
def divide1(train_x,fkey,fval):
    left=pd.DataFrame([],columns=train_x.columns)
    right=pd.DataFrame([],columns=train_x.columns)
    for ix in range(train_x.shape[0]):
        val=train_x[fkey].loc[ix]
        if val>fval:
            right=right.append(train_x.loc[ix])
        else:
            left=left.append(train_x.loc[ix])
    return left,right

In [20]:
def information1(train_x,fkey,fval):
    left,right=divide1(train_x,fkey,fval)
    if left.shape[0]==0 or right.shape[0]==0:
        return -100000
    l=left.shape[0]/train_x.shape[0]
    r=right.shape[0]/train_x.shape[0]
    inf=entropy(train_x.Survived)-((l*entropy(left.Survived)+(r*entropy(right.Survived))))
    return inf

In [21]:
for f in train_x1.columns:
    print(f)
    print(information1(train_x_clean,f,train_x_clean[f].mean()))

Pclass
0.07579362743608165
Sex
0.2176601066606142
Age
0.001158644038169343
SibSp
0.009584541813400071
Parch
0.015380754493137694
Fare
0.042140692838995464


In [22]:
class decisiontree:
    #constructor
    def __init__(self,max_depth=5,depth=0):
        self.left=None
        self.right=None
        self.fval=None
        self.fkey=None
        self.max_depth=max_depth
        self.depth=depth
        self.target=None
    def train(self,x_train):
        features = ['Pclass','Sex','Age','SibSp', 'Parch', 'Fare']
        info_gains = []
        for ix in features:
            i_gain=information1(x_train,ix,x_train[ix].mean())
            info_gains.append(i_gain)
        self.fkey=features[np.argmax(info_gains)]
        self.fval=x_train[self.fkey].mean()
        data_left,data_right=divide1(x_train,self.fkey,self.fval)
        data_left = data_left.reset_index(drop=True)
        data_right = data_right.reset_index(drop=True)
         
        #Truly a left node
        if data_left.shape[0]  == 0 or data_right.shape[0] ==0:
            if x_train.Survived.mean() >= 0.5:
                self.target = "Survive"
            else:
                self.target = "Dead"
            return
        #Stop earyly when depth >=max depth
        if(self.depth>=self.max_depth):
            if x_train.Survived.mean() >= 0.5:
                self.target = "Survive"
            else:
                self.target = "Dead"
            return
        #Recursive Case
        self.left = decisiontree(depth=self.depth+1,max_depth=self.max_depth)
        self.left.train(data_left)
        
        self.right = decisiontree(depth=self.depth+1,max_depth=self.max_depth)
        self.right.train(data_right)
        
        #You can set the target at every node
        if x_train.Survived.mean() >= 0.5:
            self.target = "Survive"
        else:
            self.target = "Dead"
        return
    def predict(self,test):
        if test[self.fkey]>self.fval:
            #go to right
            if self.right is None:
                return self.target
            return self.right.predict(test)
        else:
            if self.left is None:
                return self.target
            return self.left.predict(test)

# Train-Validation-Test Set Split¶

In [23]:
split = int(0.7*train_x_clean.shape[0])
train_data = train_x_clean[:split]
test_data = train_x_clean[split:]
test_data = test_data.reset_index(drop=True)

In [24]:
print(train_data.shape,test_data.shape)

(623, 7) (268, 7)


In [25]:
dt = decisiontree()

In [26]:
dt.train(train_data)

In [27]:
print(dt.fkey)
print(dt.fval)
print(dt.left.fkey)
print(dt.right.fkey)

Sex
0.6292134831460674
Pclass
Fare


In [28]:
y_pred = []
for ix in range(test_data.shape[0]):
    y_pred.append(dt.predict(test_data.loc[ix]))

In [29]:
le = LabelEncoder()
y_pred = le.fit_transform(y_pred)

In [30]:
print(y_pred)

[0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 1 0 0 0 0 1 0 1 0 1 1 0 0 1 0 0
 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0
 1 0 0 1 0 1 0 0 0 1 0 1 0 1 0 0 0 0 0 1 1 0 0 1 0 0 0 1 0 1 1 0 0 1 0 0 0
 0 0 0 0 0 0 0 1 1 0 0 0 1 1 1 0 1 0 0 0 1 0 0 0 0 1 0 0 0 1 0 1 0 1 0 0 0
 0 1 0 1 0 0 0 0 1 1 1 0 0 0 0 1 0 1 0 0 0 0 0 0 0 1 0 0 0 0 1 1 0 0 0 0 1
 0 1 0 0 0 0 0 0 1 0 0 0 1 0 0 1 0 0 0 0 0 1 0 1 0 0 0 1 0 0 0 0 0 0 1 0 0
 0 0 0 0 1 0 0 1 1 1 1 1 0 1 0 0 0 1 0 0 1 1 0 0 0 0 1 0 0 1 1 0 0 0 1 1 0
 0 0 0 0 0 1 0 0 0]


In [31]:
y_pred = np.array(y_pred).reshape((-1,1))

In [32]:
acc = np.sum(np.array(y_pred)==np.array(test_data[output_cols]))/y_pred.shape[0]
print(acc)

0.8171641791044776


In [33]:
test_x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Name           418 non-null object
Sex            418 non-null object
Age            332 non-null float64
SibSp          418 non-null int64
Parch          418 non-null int64
Ticket         418 non-null object
Fare           417 non-null float64
Cabin          91 non-null object
Embarked       418 non-null object
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


In [58]:
test_y=[]
for ix in range(test_x_clean.shape[0]):
    test_y.append(dt.predict(test_x_clean.loc[ix]))

In [62]:
temp=test_x['PassengerId']

In [71]:
temp=np.array(temp)
test_y=le.fit_transform(test_y)

In [83]:
test_y=test_y.reshape((-1,1))
temp=temp.reshape((-1,1))

In [88]:
x1=np.hstack((temp,test_y))
x1.shape

(418, 2)

In [89]:
dfx=pd.DataFrame(x1,columns=['PassengerID','Survived'])
dfx.to_csv('y_predicted for titanic for kaggle.csv',index=False)

# TREE IMPLEMENTATION USING SKLEARN

In [35]:
from sklearn.tree import DecisionTreeClassifier

In [228]:
sk_tree=DecisionTreeClassifier(criterion='entropy',max_depth=5) #making a object of decision tree classifier


In [229]:
sk_tree.fit(train_x_clean[input_cols],train_x_clean[output_cols])

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=5,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [230]:
x=sk_tree.predict(train_x_clean[input_cols])

In [231]:
x=x.reshape((-1,1))

In [232]:
x.shape

(891, 1)

In [233]:
sk_tree.score(train_x_clean[input_cols],train_x_clean[output_cols])

0.8473625140291807

In [234]:
x=-float((6/11)*np.log2(6/11)+(5/11)*np.log2(5/11))
x

0.9940302114769565

In [235]:
x1=-float((2/3)*np.log2(2/3)+(1/3)*np.log2(1/3))
x2=float(4/11)+float(4/11)
x2

0.7272727272727273

In [236]:
print(test_x.shape)
test_x.info()

(418, 11)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Name           418 non-null object
Sex            418 non-null object
Age            332 non-null float64
SibSp          418 non-null int64
Parch          418 non-null int64
Ticket         418 non-null object
Fare           417 non-null float64
Cabin          91 non-null object
Embarked       418 non-null object
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


In [237]:
columns_to_drop=["PassengerId","Name","Ticket","Embarked","Cabin"]
test_x_clean=test_x.drop(columns_to_drop,axis=1)
test_x_clean.head(n=10)

,Pclass,Sex,Age,SibSp,Parch,Fare
0,3,male,34.5,0,0,7.8292
1,3,female,47.0,1,0,7.0000
2,2,male,62.0,0,0,9.6875
3,3,male,27.0,0,0,8.6625
4,3,female,22.0,1,1,12.2875
5,3,male,14.0,0,0,9.2250
6,3,female,30.0,0,0,7.6292
7,2,male,26.0,1,1,29.0000
8,3,female,18.0,0,0,7.2292
9,3,male,21.0,2,0,24.1500


In [238]:
test_x_clean["Sex"]=le.fit_transform(test_x_clean["Sex"])

In [239]:
test_x_clean.head(n=10)
test_x_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 6 columns):
Pclass    418 non-null int64
Sex       418 non-null int32
Age       332 non-null float64
SibSp     418 non-null int64
Parch     418 non-null int64
Fare      417 non-null float64
dtypes: float64(2), int32(1), int64(3)
memory usage: 18.0 KB


In [240]:
print(test_x_clean.shape)

(418, 6)


In [241]:
test_x_clean=test_x_clean.fillna(test_x_clean["Age"].mean())

In [242]:
test_x_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 6 columns):
Pclass    418 non-null int64
Sex       418 non-null int32
Age       418 non-null float64
SibSp     418 non-null int64
Parch     418 non-null int64
Fare      418 non-null float64
dtypes: float64(2), int32(1), int64(3)
memory usage: 18.0 KB


In [243]:
test_y=sk_tree.predict(test_x_clean)

In [244]:
print(test_y)

[0 0 0 0 1 0 1 0 1 0 0 0 1 0 1 1 0 0 1 0 0 1 1 0 1 0 1 0 0 0 0 0 1 0 0 0 1
 1 0 0 0 0 0 1 1 0 0 0 1 1 0 0 1 1 0 0 0 0 0 1 0 0 0 1 1 1 1 0 0 1 1 0 1 0
 1 0 0 1 0 1 1 0 0 0 0 0 1 1 1 1 1 0 1 0 0 0 1 0 1 0 1 0 0 0 1 0 0 0 0 0 0
 1 1 1 1 0 0 1 0 1 1 0 1 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0
 0 0 1 0 0 1 0 0 1 1 0 1 1 1 1 0 0 1 0 0 1 1 0 0 0 0 0 1 1 0 1 1 0 0 1 0 1
 0 1 0 0 0 0 0 1 0 1 0 1 1 0 1 1 1 0 1 0 0 1 0 1 0 0 0 0 1 0 0 1 0 1 0 1 0
 1 0 1 1 0 1 0 0 0 1 0 0 0 0 0 0 1 1 1 1 0 0 0 0 1 0 1 1 1 0 0 0 0 0 0 0 1
 0 0 0 1 1 0 0 0 0 1 0 0 0 1 1 0 1 0 0 0 0 1 1 1 1 1 0 0 0 0 0 0 1 0 0 0 0
 0 0 0 0 0 0 0 0 1 1 0 1 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 1 0 1 0 0 0 1 0 0
 1 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 1 1 0 0 0 1 0 1 0 0 1 0 1 1 0 1 0 0 1 1 0
 0 1 0 0 1 1 1 0 0 0 0 0 1 1 0 1 0 0 0 0 0 1 1 0 0 1 0 1 0 0 1 0 1 0 0 0 0
 0 1 1 1 1 1 0 1 0 0 0]


In [245]:
t=pd.read_csv("Test.csv")
t1=pd.read_csv("Train.csv")

In [246]:
print(t.shape)
print(t1.shape)

(300, 13)
(1009, 14)


In [247]:
t1.head(n=10)

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,3.0,0.0,"O'Donoghue, Ms. Bridget",female,NaN,0.0,0.0,364856,7.7500,NaN,Q,NaN,NaN,NaN
1,2.0,0.0,"Morley, Mr. Henry Samuel (""Mr Henry Marshall"")",male,39.0,0.0,0.0,250655,26.0000,NaN,S,NaN,NaN,NaN
2,2.0,1.0,"Smith, Miss. Marion Elsie",female,40.0,0.0,0.0,31418,13.0000,NaN,S,9,NaN,NaN
3,3.0,1.0,"Goldsmith, Mrs. Frank John (Emily Alice Brown)",female,31.0,1.0,1.0,363291,20.5250,NaN,S,C D,NaN,"Strood, Kent, England Detroit, MI"
4,3.0,1.0,"McCoy, Miss. Agnes",female,NaN,2.0,0.0,367226,23.2500,NaN,Q,16,NaN,NaN
5,2.0,0.0,"Gaskell, Mr. Alfred",male,16.0,0.0,0.0,239865,26.0000,NaN,S,NaN,NaN,"Liverpool / Montreal, PQ"
6,2.0,0.0,"Phillips, Mr. Escott Robert",male,43.0,0.0,1.0,S.O./P.P. 2,21.0000,NaN,S,NaN,NaN,"Ilfracombe, Devon"
7,1.0,1.0,"Leader, Dr. Alice (Farnham)",female,49.0,0.0,0.0,17465,25.9292,D17,S,8,NaN,"New York, NY"
8,1.0,0.0,"Brandeis, Mr. Emil",male,48.0,0.0,0.0,PC 17591,50.4958,B10,C,NaN,208.0,"Omaha, NE"
9,2.0,0.0,"Wheeler, Mr. Edwin ""Frederick""",male,NaN,0.0,0.0,SC/PARIS 2159,12.8750,NaN,S,NaN,NaN,NaN


In [248]:
train_x.head(n=10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


In [249]:
t1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1009 entries, 0 to 1008
Data columns (total 14 columns):
pclass       1009 non-null float64
survived     1009 non-null float64
name         1009 non-null object
sex          1009 non-null object
age          812 non-null float64
sibsp        1009 non-null float64
parch        1009 non-null float64
ticket       1009 non-null object
fare         1008 non-null float64
cabin        229 non-null object
embarked     1008 non-null object
boat         374 non-null object
body         98 non-null float64
home.dest    582 non-null object
dtypes: float64(7), object(7)
memory usage: 110.4+ KB


In [250]:
t1_clean=t1.drop(["name","embarked","boat","home.dest","body","ticket","cabin"],axis=1)

In [251]:
t1_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1009 entries, 0 to 1008
Data columns (total 7 columns):
pclass      1009 non-null float64
survived    1009 non-null float64
sex         1009 non-null object
age         812 non-null float64
sibsp       1009 non-null float64
parch       1009 non-null float64
fare        1008 non-null float64
dtypes: float64(6), object(1)
memory usage: 55.3+ KB


In [252]:
t1_clean=t1_clean.fillna(t1_clean["age"].mean())

In [253]:
t1_clean["sex"]=le.fit_transform(t1_clean["sex"])

In [260]:
sk_tree1=DecisionTreeClassifier(criterion='entropy',max_depth=9)
input1=["pclass","sex","age","sibsp","parch","fare"]
output1=["survived"]
sk_tree1.fit(t1_clean[input1],t1_clean[output1])

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=9,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [261]:
sk_tree1.score(t1_clean[input1],t1_clean[output1])

0.8790882061446977

In [262]:
t.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 13 columns):
pclass       300 non-null float64
name         300 non-null object
sex          300 non-null object
age          234 non-null float64
sibsp        300 non-null float64
parch        300 non-null float64
ticket       300 non-null object
fare         300 non-null float64
cabin        66 non-null object
embarked     299 non-null object
boat         112 non-null object
body         23 non-null float64
home.dest    163 non-null object
dtypes: float64(6), object(7)
memory usage: 30.5+ KB


In [311]:
t_clean=t.drop(["name","home.dest","boat","body","ticket","cabin"],axis=1)
t_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 7 columns):
pclass      300 non-null float64
sex         300 non-null object
age         234 non-null float64
sibsp       300 non-null float64
parch       300 non-null float64
fare        300 non-null float64
embarked    299 non-null object
dtypes: float64(5), object(2)
memory usage: 16.5+ KB


In [313]:
t_clean=t_clean.fillna(t_clean["age"].mean())
t_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 7 columns):
pclass      300 non-null float64
sex         300 non-null object
age         300 non-null float64
sibsp       300 non-null float64
parch       300 non-null float64
fare        300 non-null float64
embarked    300 non-null object
dtypes: float64(5), object(2)
memory usage: 16.5+ KB


In [318]:
t_clean["sex"]=le.fit_transform(t_clean["sex"])
t_clean.head(n=10)

,pclass,sex,age,sibsp,parch,fare,embarked
0,1.0,1,36.000000,0.0,0.0,26.3875,S
1,3.0,0,30.027422,8.0,2.0,69.5500,S
2,1.0,1,30.027422,0.0,0.0,50.0000,S
3,2.0,1,34.000000,0.0,0.0,13.0000,S
4,2.0,1,28.000000,0.0,0.0,13.0000,S
5,2.0,0,31.000000,1.0,1.0,26.2500,S
6,1.0,1,40.000000,0.0,0.0,27.7208,C
7,3.0,1,30.027422,0.0,0.0,7.2292,C
8,3.0,1,22.000000,0.0,0.0,8.0500,S
9,1.0,1,30.000000,0.0,0.0,45.5000,S


In [323]:
t_clean.info()
from sklearn.tree import DecisionTreeClassifier
sk_tree1=DecisionTreeClassifier()
sk_tree1.fit(t_clean[input])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 7 columns):
pclass      300 non-null float64
sex         300 non-null int64
age         300 non-null float64
sibsp       300 non-null float64
parch       300 non-null float64
fare        300 non-null float64
embarked    300 non-null object
dtypes: float64(5), int64(1), object(1)
memory usage: 16.5+ KB


In [324]:
t_y=sk_tree1.predict(t_clean)

NotFittedError: This DecisionTreeClassifier instance is not fitted yet. Call 'fit' with appropriate arguments before using this method.

In [268]:
print(type(t_y))

<class 'numpy.ndarray'>


In [269]:
t_y=t_y.reshape((-1,1))

In [270]:
t_y.shape

(300, 1)

In [271]:
df1=pd.DataFrame(data=t_y,columns=["Outcome"])
df1.to_csv('y_prediction titanic survivor.csv',index=True)

# RANDOM FOREST

In [90]:
from sklearn.ensemble import RandomForestClassifier

In [91]:
rf=RandomForestClassifier(n_estimators=22,criterion='entropy',max_depth=3)

In [92]:
train_x_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 7 columns):
Survived    891 non-null int64
Pclass      891 non-null int64
Sex         891 non-null int32
Age         714 non-null float64
SibSp       891 non-null int64
Parch       891 non-null int64
Fare        891 non-null float64
dtypes: float64(2), int32(1), int64(4)
memory usage: 45.3 KB


In [93]:
rf.fit(train_x_clean[input_cols],train_x_clean[output_cols])

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

In [304]:
y=rf.predict(t_clean)

In [305]:
y=y.reshape((-1,1))

In [306]:
df1=pd.DataFrame(data=y,columns=["Outcome"])
df1.to_csv('y_prediction titanic1 survivor.csv',index=True)

In [307]:
rf.score(t1_clean[input1],t1_clean[output1])

0.8067393458870169

In [326]:
input11=["pclass","sex","age","sibsp","parch","fare","embarked"]

In [328]:
t_clean.head(n=10)

,pclass,sex,age,sibsp,parch,fare,embarked
0,1.0,1,36.000000,0.0,0.0,26.3875,S
1,3.0,0,30.027422,8.0,2.0,69.5500,S
2,1.0,1,30.027422,0.0,0.0,50.0000,S
3,2.0,1,34.000000,0.0,0.0,13.0000,S
4,2.0,1,28.000000,0.0,0.0,13.0000,S
5,2.0,0,31.000000,1.0,1.0,26.2500,S
6,1.0,1,40.000000,0.0,0.0,27.7208,C
7,3.0,1,30.027422,0.0,0.0,7.2292,C
8,3.0,1,22.000000,0.0,0.0,8.0500,S
9,1.0,1,30.000000,0.0,0.0,45.5000,S


In [329]:
t_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 7 columns):
pclass      300 non-null float64
sex         300 non-null int64
age         300 non-null float64
sibsp       300 non-null float64
parch       300 non-null float64
fare        300 non-null float64
embarked    300 non-null object
dtypes: float64(5), int64(1), object(1)
memory usage: 16.5+ KB


In [330]:
t1_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1009 entries, 0 to 1008
Data columns (total 7 columns):
pclass      1009 non-null float64
survived    1009 non-null float64
sex         1009 non-null int32
age         1009 non-null float64
sibsp       1009 non-null float64
parch       1009 non-null float64
fare        1009 non-null float64
dtypes: float64(6), int32(1)
memory usage: 51.3 KB


In [333]:
t1=t1.drop(["name","home.dest","boat","body","ticket","cabin"],axis=1)

In [334]:
t1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1009 entries, 0 to 1008
Data columns (total 8 columns):
pclass      1009 non-null float64
survived    1009 non-null float64
sex         1009 non-null object
age         812 non-null float64
sibsp       1009 non-null float64
parch       1009 non-null float64
fare        1008 non-null float64
embarked    1008 non-null object
dtypes: float64(6), object(2)
memory usage: 63.1+ KB


In [340]:
t1_clean=t1.fillna(t1["age"].mean())
t1_clean=t1.fillna(t1["embarked"].mode())
t1_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1009 entries, 0 to 1008
Data columns (total 8 columns):
pclass      1009 non-null float64
survived    1009 non-null float64
sex         1009 non-null object
age         812 non-null float64
sibsp       1009 non-null float64
parch       1009 non-null float64
fare        1008 non-null float64
embarked    1008 non-null object
dtypes: float64(6), object(2)
memory usage: 63.1+ KB


In [341]:
t1_clean["sex"]=le.fit_transform(t1_clean["sex"])
t1_clean["embarked"]=le.fit
t1_clean.head(n=10)

,pclass,survived,sex,age,sibsp,parch,fare,embarked
0,3.0,0.0,0,NaN,0.0,0.0,7.7500,Q
1,2.0,0.0,1,39.0,0.0,0.0,26.0000,S
2,2.0,1.0,0,40.0,0.0,0.0,13.0000,S
3,3.0,1.0,0,31.0,1.0,1.0,20.5250,S
4,3.0,1.0,0,NaN,2.0,0.0,23.2500,Q
5,2.0,0.0,1,16.0,0.0,0.0,26.0000,S
6,2.0,0.0,1,43.0,0.0,1.0,21.0000,S
7,1.0,1.0,0,49.0,0.0,0.0,25.9292,S
8,1.0,0.0,1,48.0,0.0,0.0,50.4958,C
9,2.0,0.0,1,NaN,0.0,0.0,12.8750,S


In [342]:
rf.fit(t1_clean[input11],t1_clean[output1])

ValueError: could not convert string to float: 'Q'